In [ ]:
w# as you can see current ram is 12GB only, to increase it run below code

# Data Required Packages and Uploads

In [ ]:
pip install sh

     |████████████████████████████████| 40kB 3.3MB/s 


In [ ]:
!apt-get install bedtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  bedtools
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 577 kB of archives.
After this operation, 2,040 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 bedtools amd64 2.26.0+dfsg-5 [577 kB]
Fetched 577 kB in 1s (752 kB/s)
Selecting previously unselected package bedtools.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../bedtools_2.26.0+dfsg-5_amd64.deb ...
Unpacking bedtools (2.26.0+dfsg-5) ...
Setting up bedtools (2.26.0+dfsg-5) ...


In [ ]:
from google.colab import files
import pandas as pd
from itertools import islice
from sh import gunzip
import numpy as np
import os

To grant access for reaching google Drive folders.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To upload and unzip a file from local to google colab.

In [ ]:
#uploaded2 = files.upload()

In [ ]:
#gunzip('/content/Homo_sapiens.GRCh37.87.gtf.gz')

# Read File and edit columns

In [ ]:
fl = open("/content/drive/MyDrive/Scripts/Data/Homo_sapiens.GRCh37.87.gtf","r")

level_I_genes = []
level_II_genes = []


for line in fl:
  column = line.split(" ")
  chromosome = column[0]
  feature = column[2]
  start = column[3]
  end = column[4]
  strand = column[6]
  gene_name = column[13]

  if feature == "gene" and (int(end) - int(start) >= 10000):        # Checks gene length
    level_I_genes.append(column)

fl.close()

# Data Frame creation

In [ ]:
col_headers = ["Chromosome_Name", "Sequence_Source", "Sequence_Feature", "Start", "End", "Strand", "Gene_ID", "Gene_Name"]
List_b_DF = []

for EL in level_I_genes:
  Chromosome_Name = EL[0]
  Sequence_Source = EL[1]
  Sequence_Feature = EL[2]
  Start = EL[3]
  End = EL[4]
  Strand = EL[6]
  Gene_ID = EL[9]
  Gene_Name = EL[13]

  List_b_DF.append([Chromosome_Name, Sequence_Source, Sequence_Feature, Start, End, Strand, Gene_ID, Gene_Name])
  DF = pd.DataFrame(List_b_DF, columns = col_headers)


DF

,Chromosome_Name,Sequence_Source,Sequence_Feature,Start,End,Strand,Gene_ID,Gene_Name
0,1,ensembl_havana,gene,14363,29806,-,"""ENSG00000227232""","""WASH7P"""
1,1,havana,gene,89295,133566,-,"""ENSG00000238009""","""RP11-34P13.7"""
2,1,havana,gene,141474,173862,-,"""ENSG00000241860""","""RP11-34P13.13"""
3,1,havana,gene,227615,267253,-,"""ENSG00000228463""","""AP006222.2"""
4,1,ensembl_havana,gene,317720,453948,+,"""ENSG00000237094""","""RP4-669L17.10"""
...,...,...,...,...,...,...,...,...
20701,GL000218.1,ensembl,gene,38792,97421,-,"""ENSG00000215615""","""AL354822.1"""
20702,GL000209.1,ensembl,gene,7891,96246,+,"""ENSG00000215764""","""KIR2DL2"""
20703,GL000191.1,ensembl,gene,36277,50281,-,"""ENSG00000215699""","""SRSF10"""
20704,GL000242.1,ensembl,gene,12836,34543,+,"""ENSG00000251180""","""CU459201.1"""


# DataFrame filtering steps

Eleminating genes which span smaller than 10000 bases

In [ ]:
convert_dict = {"Chromosome_Name" : str,
                "Sequence_Source" : str,
                "Sequence_Feature" : str,
                "Start" : int,
                "End" : int,
                "Strand" : str,
                "Gene_ID" : str,
                "Gene_Name" : str} 
  
DF = DF.astype(convert_dict) 
#DF.tail()

Candidate_index = DF[DF["Start"] <= 10000].index
#print(Candidate_index)

DF = DF.drop(Candidate_index)

DF[DF["Start"] <= 15000]           # To check whether the row is discarded

,Chromosome_Name,Sequence_Source,Sequence_Feature,Start,End,Strand,Gene_ID,Gene_Name
0,1,ensembl_havana,gene,14363,29806,-,"""ENSG00000227232""","""WASH7P"""
10144,9,ensembl_havana,gene,14511,29739,-,"""ENSG00000181404""","""XXyac-YRM2039.2"""
20704,GL000242.1,ensembl,gene,12836,34543,+,"""ENSG00000251180""","""CU459201.1"""


In [ ]:
DF[DF["Start"] < 10]

,Chromosome_Name,Sequence_Source,Sequence_Feature,Start,End,Strand,Gene_ID,Gene_Name


In [ ]:
DF[(DF["End"] - DF["Start"]) == 10000]

,Chromosome_Name,Sequence_Source,Sequence_Feature,Start,End,Strand,Gene_ID,Gene_Name
19773,19,ensembl_havana,gene,55417508,55427508,+,"""ENSG00000189430""","""NCR1"""


Eleminating genes found in nontraditional chromosomes

In [ ]:
Trash_DF = DF[DF["Chromosome_Name"].str.match('(^G.+)') == True].index
print(Trash_DF)

DF = DF.drop(Trash_DF)
DF.tail()

# Second Filtering Step

Eleminating gene overlaps considering strandedness

In [ ]:
fl2 = DF.copy()

fl2

# Gene Overlap Method - Strand Exclusive Version

In [ ]:
def GeneOverlap(fl2):
  Trash_list = []

  fl2_chr_list = fl2["Chromosome_Name"].tolist()
  fl2_start_list = fl2["Start"].tolist()
  fl2_end_list = fl2["End"].tolist()

  for i in range(0, len(fl2)-1):
    for j in range(i+1, len(fl2)):

      if fl2_chr_list[i] == "1" and fl2_chr_list[j] == "1":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            
    
      if fl2_chr_list[i] == "2" and fl2_chr_list[j] == "2":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "3" and fl2_chr_list[j] == "3":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "4" and fl2_chr_list[j] == "4":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "5" and fl2_chr_list[j] == "5":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "6" and fl2_chr_list[j] == "6":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "7" and fl2_chr_list[j] == "7":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "X" and fl2_chr_list[j] == "X":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "8" and fl2_chr_list[j] == "8":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "9" and fl2_chr_list[j] == "9":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "10" and fl2_chr_list[j] == "10":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "11" and fl2_chr_list[j] == "11":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "12" and fl2_chr_list[j] == "12":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "13" and fl2_chr_list[j] == "13":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "14" and fl2_chr_list[j] == "14":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "15" and fl2_chr_list[j] == "15":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "16" and fl2_chr_list[j] == "16":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "17" and fl2_chr_list[j] == "17":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "18" and fl2_chr_list[j] == "18":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "20" and fl2_chr_list[j] == "20":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "19" and fl2_chr_list[j] == "19":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "Y" and fl2_chr_list[j] == "Y":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "22" and fl2_chr_list[j] == "22":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
            

      if fl2_chr_list[i] == "21" and fl2_chr_list[j] == "21":
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)


  return Trash_list

# Gene Overlap Method - Strand Specific Version

In [ ]:
def GeneOverlap(fl2):
  Trash_list = []

  fl2_chr_list = fl2["Chromosome_Name"].tolist()
  fl2_start_list = fl2["Start"].tolist()
  fl2_end_list = fl2["End"].tolist()
  fl2_strand_list = fl2["Strand"].tolist()

  for i in range(0, len(fl2)- 1):
    for j in range(i+1, len(fl2)):

      if fl2_chr_list[i] == "1" and fl2_chr_list[j] == "1" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)
    
      if fl2_chr_list[i] == "2" and fl2_chr_list[j] == "2" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "3" and fl2_chr_list[j] == "3" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "4" and fl2_chr_list[j] == "4" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "5" and fl2_chr_list[j] == "5" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "6" and fl2_chr_list[j] == "6" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "7" and fl2_chr_list[j] == "7" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "X" and fl2_chr_list[j] == "X" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "8" and fl2_chr_list[j] == "8" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "9" and fl2_chr_list[j] == "9" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "10" and fl2_chr_list[j] == "10" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "11" and fl2_chr_list[j] == "11" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "12" and fl2_chr_list[j] == "12" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "13" and fl2_chr_list[j] == "13" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "14" and fl2_chr_list[j] == "14" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "15" and fl2_chr_list[j] == "15" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "16" and fl2_chr_list[j] == "16" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "17" and fl2_chr_list[j] == "17" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "18" and fl2_chr_list[j] == "18" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "20" and fl2_chr_list[j] == "20" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "19" and fl2_chr_list[j] == "19" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "Y" and fl2_chr_list[j] == "Y" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "22" and fl2_chr_list[j] == "22" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

      if fl2_chr_list[i] == "21" and fl2_chr_list[j] == "21" and fl2_strand_list[i] == fl2_strand_list[j]:
        if (fl2_start_list[i] <= fl2_end_list[j]) and (fl2_end_list[i] >= fl2_start_list[j]):
          if i not in Trash_list:
            Trash_list.append(i)

  return Trash_list

# Check GeneOverlap results

In [ ]:
fl2["Chromosome_Name"].unique()

In [ ]:
OverlappedGenes = GeneOverlap(fl2)
print(OverlappedGenes)

In [ ]:
print(len(OverlappedGenes))

In [ ]:
fl2.index

In [ ]:
fl2 = fl2.drop(OverlappedGenes)
fl2

In [ ]:
fl2.index

In [ ]:
DF4 = fl2.copy()
DF4

In [ ]:
DF4.loc[602:603]

In [ ]:
del OverlappedGenes

# Selecting genes having at least (set distance to 15 000) 10000 base distance apart

In [ ]:
Plus_DF = DF4[DF4["Strand"] == "+"]
Minus_DF = DF4[DF4["Strand"] == "-"]

In [ ]:
Plus_DF["Chromosome_Name_shifted"] = Plus_DF["Chromosome_Name"].shift(-1)
Plus_DF["Start_shifted"] = Plus_DF["Start"].shift(-1)
Plus_DF["End_shifted"] = Plus_DF["End"].shift(-1)

In [ ]:
Minus_DF["Chromosome_Name_shifted"] = Minus_DF["Chromosome_Name"].shift(-1)
Minus_DF["Start_shifted"] = Minus_DF["Start"].shift(-1)
Minus_DF["End_shifted"] = Minus_DF["End"].shift(-1)

In [ ]:
Minus_DF

In [ ]:
Plus_DF

For Plus Strand

In [ ]:
CHR_intersections_list = Plus_DF[Plus_DF["Chromosome_Name"] != Plus_DF["Chromosome_Name_shifted"]].index
print(CHR_intersections_list)
print(len(CHR_intersections_list))

In [ ]:
Plus_DF["Chromosome_Name"].unique()

In [ ]:
CHR_list = ['1', '2', '3', '4', '5', '6', '7', 'X', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '20', '19', 'Y', '22', '21']

for name, indexx in zip(CHR_list, CHR_intersections_list):
  Plus_DF.loc[indexx, ["Chromosome_Name_shifted", "Start_shifted", "End_shifted"]] =  [name, 9999999999, 9999999999]

In [ ]:
Plus_DF[Plus_DF["Chromosome_Name"] != Plus_DF["Chromosome_Name_shifted"]].index

In [ ]:
Plus_DF

In [ ]:
Plus_GeneDiff_index = []
c = 0

for _, CHR_DF in Plus_DF.groupby("Chromosome_Name"):
  GeneDiff_index = CHR_DF[abs(CHR_DF["End"] - CHR_DF["Start_shifted"]) <= 15000].index
  for index_el in GeneDiff_index:
    Plus_GeneDiff_index.append(index_el)
    c += 1

print(c)

In [ ]:
print(Plus_GeneDiff_index)

In [ ]:
print(len(Plus_GeneDiff_index))

1069


In [ ]:
Plus_Diff_DF = Plus_DF.drop(Plus_GeneDiff_index)

Plus_Diff_DF

In [ ]:
Plus_Diff_DF[Plus_Diff_DF["Chromosome_Name"] != Plus_Diff_DF["Chromosome_Name_shifted"]].index

Int64Index([], dtype='int64')

In [ ]:
del CHR_intersections_list, CHR_list, name, indexx, _, CHR_DF, GeneDiff_index, index_el, c, Plus_GeneDiff_index

For Minus Strand

In [ ]:
CHR_intersections_list = Minus_DF[Minus_DF["Chromosome_Name"] != Minus_DF["Chromosome_Name_shifted"]].index
print(CHR_intersections_list)
print(len(CHR_intersections_list))

In [ ]:
CHR_list = ['1', '2', '3', '4', '5', '6', '7', 'X', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '20', '19', 'Y', '22', '21']

for name, indexx in zip(CHR_list, CHR_intersections_list):
  Minus_DF.loc[indexx, ["Chromosome_Name_shifted", "Start_shifted", "End_shifted"]] =  [name, 9999999999, 9999999999]

In [ ]:
Minus_DF[Minus_DF["Chromosome_Name"] != Minus_DF["Chromosome_Name_shifted"]].index

In [ ]:
Minus_DF

In [ ]:
Minus_GeneDiff_index = []
c = 0

for _, CHR_DF in Minus_DF.groupby("Chromosome_Name"):
  GeneDiff_index = CHR_DF[abs(CHR_DF["End"] - CHR_DF["Start_shifted"]) <= 15000].index
  for index_el in GeneDiff_index:
    Minus_GeneDiff_index.append(index_el)
    c += 1

print(c)

1037


In [ ]:
Minus_Diff_DF = Minus_DF.drop(Minus_GeneDiff_index)

Minus_Diff_DF

In [ ]:
Minus_Diff_DF[Minus_Diff_DF["Chromosome_Name"] != Minus_Diff_DF["Chromosome_Name_shifted"]].index

In [ ]:
del CHR_intersections_list, CHR_list, name, indexx, _, CHR_DF, GeneDiff_index, index_el, c, Minus_GeneDiff_index

In [ ]:
Minus_Diff_DF["Chromosome_Name"].unique()

In [ ]:
Minus_Diff_DF.loc[6633]     # 18176, 18687, 19887, 20003, 20439, 20688

# Merge Plus and Minus stranded Genes together

In [ ]:
Filtered_DF = pd.concat([Plus_Diff_DF, Minus_Diff_DF], ignore_index=False, sort=False)

In [ ]:
Filtered_DF

In [ ]:
del Filtered_DF["Chromosome_Name_shifted"], Filtered_DF["Start_shifted"], Filtered_DF["End_shifted"]

In [ ]:
Filtered_DF

In [ ]:
Filtered_sorted_DF = Filtered_DF.sort_index()

In [ ]:
Filtered_sorted_DF = Filtered_sorted_DF.reset_index(drop=True)

In [ ]:
Filtered_sorted_DF

# Check Gene Overlaps agan in the new DataFrame

In [ ]:
OverlappedGenes = GeneOverlap(Filtered_sorted_DF)
print(OverlappedGenes)

[11105, 11107, 11116, 11120, 11127, 11129, 11131, 11136, 11146, 11156, 11157, 11167, 11171, 11178, 11181, 11185, 11192, 11197, 11202, 11205, 11207, 11221, 11225, 11227, 11232, 11237, 11240, 11248, 11252, 11261, 11273, 11277, 11281, 11291, 11297, 11303, 11305, 11311, 11313, 11323, 11329, 11346, 11353, 11355, 11358, 11362, 11365, 11369, 11387, 11393, 11402, 11405, 11408, 11411, 11421, 11424, 11432, 11444, 11447, 11452, 11455, 11457, 11460, 11462, 11464, 11468, 11471, 11473, 11479, 11484, 11490, 11498, 11500, 11505, 11517, 11525, 11531, 11533, 11541, 11545, 11549, 11557, 11559, 11562, 11572, 11575, 11577, 11580, 11589, 11595, 11607, 11609, 11612, 11620, 11622, 11625, 11644, 11649, 11656, 11666, 11668, 11674, 11678, 11682, 11692, 11696, 11701, 11705, 11708, 11713, 11715, 11720, 11727, 11731, 11737, 11739, 11744, 11749, 11751, 11754, 11759, 11763, 11776, 11778, 11781, 11786, 11789, 11795, 11807, 11818, 11834, 11845, 11847, 11850, 11852, 11856, 11858, 11862, 11869, 11882, 11884, 11888, 11894

In [ ]:
print(len(OverlappedGenes))

In [ ]:
Filtered_sorted_DF.index

RangeIndex(start=0, stop=13554, step=1)

In [ ]:
del OverlappedGenes

Beware! Do not use below section if there is no overlapped genes at 

```
OverlappedGenes list.
```


In [ ]:
Filtered_sorted_DF = Filtered_sorted_DF.drop(OverlappedGenes)
Filtered_sorted_DF

,Chromosome_Name,Sequence_Source,Sequence_Feature,Start,End,Strand,Gene_ID,Gene_Name
0,1,ensembl_havana,gene,14363,29806,-,"""ENSG00000227232""","""WASH7P"""
1,1,havana,gene,141474,173862,-,"""ENSG00000241860""","""RP11-34P13.13"""
2,1,havana,gene,227615,267253,-,"""ENSG00000228463""","""AP006222.2"""
3,1,ensembl_havana,gene,317720,453948,+,"""ENSG00000237094""","""RP4-669L17.10"""
4,1,havana,gene,536816,659930,-,"""ENSG00000230021""","""RP5-857K21.4"""
...,...,...,...,...,...,...,...,...
13549,21,ensembl_havana,gene,47649131,47679304,+,"""ENSG00000215424""","""MCM3AP-AS1"""
13550,21,ensembl_havana,gene,47706251,47717665,+,"""ENSG00000182362""","""YBEY"""
13551,21,ensembl_havana,gene,47720095,47743789,-,"""ENSG00000160298""","""C21orf58"""
13552,21,ensembl_havana,gene,47878812,47989926,+,"""ENSG00000160305""","""DIP2A"""


In [ ]:
Filtered_sorted_DF.index

# Write _Filtered_sorted_DF_ as a csv file.

In [ ]:
Filtered_sorted_DF

In [ ]:
#Filtered_sorted_DF.str.replace({'Gene_ID' : '"', 'Gene_Name' : '"'}, '')

In [ ]:
Filtered_sorted_DF["Gene_Name"] = Filtered_sorted_DF["Gene_Name"].str.replace('"','')

In [ ]:
Filtered_sorted_DF["Gene_ID"] = Filtered_sorted_DF["Gene_ID"].str.replace('"','')

In [ ]:
Filtered_sorted_DF

In [ ]:
Filtered_sorted_DF.to_csv("/content/drive/MyDrive/Scripts/Data/Filtered_sorted_WGS_GeneAnnotation.csv", sep='\t', header = col_headers , index=False)

# For creating files:

1) 5kb Upstream of TSS

2) 5kb Downstream of TSS

3) 5kb Upstream of TES

4) Intergenics

## For 5kb Upstream TSS

Data Structure should look like:

Chr#   (TSS-5000)   TSS   +    .   .   .

Chr#   TES   (TES + 5000)   -   .   .   .

In [ ]:
Plus_filtered_DF = Filtered_sorted_DF[(Filtered_sorted_DF["Strand"] == "+")]
Minus_filtered_DF = Filtered_sorted_DF[(Filtered_sorted_DF["Strand"] == "-")]

In [ ]:
Filtered_sorted_DF

In [ ]:
Plus_filtered_DF

In [ ]:
Minus_filtered_DF

In [ ]:
Plus_Nstart_col = pd.to_numeric(Plus_filtered_DF["Start"]) - 5000

In [ ]:
Plus_Nstart_col

In [ ]:
Minus_Nend_col = pd.to_numeric(Minus_filtered_DF["End"]) + 5000

In [ ]:
Minus_Nend_col

Edit column values

In [ ]:
Plus_filtered_DF["End"] = Plus_filtered_DF["Start"]
Plus_filtered_DF["Start"] = Plus_Nstart_col
Plus_filtered_DF

In [ ]:
Minus_filtered_DF["Start"] = Minus_filtered_DF["End"]
Minus_filtered_DF["End"] = Minus_Nend_col
Minus_filtered_DF

Merge Plus and Minus Dataframes

In [ ]:
UTSS_DF = pd.concat([Plus_filtered_DF, Minus_filtered_DF], ignore_index=False, sort=False)

UTSS_sorted_DF = UTSS_DF.sort_index()
UTSS_sorted_DF

,Chromosome_Name,Sequence_Source,Sequence_Feature,Start,End,Strand,Gene_ID,Gene_Name
0,1,ensembl_havana,gene,29806,34806,-,ENSG00000227232,WASH7P
1,1,havana,gene,173862,178862,-,ENSG00000241860,RP11-34P13.13
2,1,havana,gene,267253,272253,-,ENSG00000228463,AP006222.2
3,1,ensembl_havana,gene,312720,317720,+,ENSG00000237094,RP4-669L17.10
4,1,havana,gene,659930,664930,-,ENSG00000230021,RP5-857K21.4
...,...,...,...,...,...,...,...,...
13549,21,ensembl_havana,gene,47644131,47649131,+,ENSG00000215424,MCM3AP-AS1
13550,21,ensembl_havana,gene,47701251,47706251,+,ENSG00000182362,YBEY
13551,21,ensembl_havana,gene,47743789,47748789,-,ENSG00000160298,C21orf58
13552,21,ensembl_havana,gene,47873812,47878812,+,ENSG00000160305,DIP2A


In [ ]:
del Plus_filtered_DF, Plus_Nstart_col, Minus_filtered_DF, Minus_Nend_col, UTSS_DF

## For 5kb Downstream TSS

Data Structure should look like:

Chr#   TSS   (TSS + 5000)   +   .   .   .

Chr#   (TES - 5000)   TES   -   .   .   .

In [ ]:
Plus_filtered_DF = Filtered_sorted_DF[(Filtered_sorted_DF["Strand"] == "+")]
Minus_filtered_DF = Filtered_sorted_DF[(Filtered_sorted_DF["Strand"] == "-")]

In [ ]:
Filtered_sorted_DF

In [ ]:
Plus_Nstart_col = pd.to_numeric(Plus_filtered_DF["Start"]) + 5000

In [ ]:
Plus_Nstart_col

In [ ]:
Minus_Nend_col = pd.to_numeric(Minus_filtered_DF["End"]) - 5000

In [ ]:
Minus_Nend_col

Edit column values

In [ ]:
Plus_filtered_DF["End"] = Plus_Nstart_col
Plus_filtered_DF

In [ ]:
Minus_filtered_DF["Start"] = Minus_Nend_col
Minus_filtered_DF

Merge Plus and Minus Dataframes

In [ ]:
DTSS_DF = pd.concat([Plus_filtered_DF, Minus_filtered_DF], ignore_index=False, sort=False)

DTSS_sorted_DF = DTSS_DF.sort_index()
DTSS_sorted_DF

In [ ]:
del Plus_filtered_DF, Plus_Nstart_col, Minus_filtered_DF, Minus_Nend_col, DTSS_DF

## For 5kb Upstream TES

Data Structure should look like:

Chr#   TES - 5000   TES   +   .   .   .

Chr#   TSS  TSS + 5000   -   .   .   .

In [ ]:
Plus_filtered_DF = Filtered_sorted_DF[(Filtered_sorted_DF["Strand"] == "+")]
Minus_filtered_DF = Filtered_sorted_DF[(Filtered_sorted_DF["Strand"] == "-")]

In [ ]:
Filtered_sorted_DF

In [ ]:
Plus_Nend_col = pd.to_numeric(Plus_filtered_DF["End"]) - 5000

In [ ]:
Plus_Nend_col

In [ ]:
Minus_Nstart_col = pd.to_numeric(Minus_filtered_DF["Start"]) + 5000

In [ ]:
Minus_Nstart_col

Edit column values

In [ ]:
Plus_filtered_DF["Start"] = Plus_Nend_col
Plus_filtered_DF

In [ ]:
Minus_filtered_DF["End"] = Minus_Nstart_col
Minus_filtered_DF

Merge Plus and Minus Dataframes

In [ ]:
UTES_DF = pd.concat([Plus_filtered_DF, Minus_filtered_DF], ignore_index=False, sort=False)

UTES_sorted_DF = UTES_DF.sort_index()
UTES_sorted_DF

,Chromosome_Name,Sequence_Source,Sequence_Feature,Start,End,Strand,Gene_ID,Gene_Name
0,1,ensembl_havana,gene,14363,19363,-,ENSG00000227232,WASH7P
1,1,havana,gene,141474,146474,-,ENSG00000241860,RP11-34P13.13
2,1,havana,gene,227615,232615,-,ENSG00000228463,AP006222.2
3,1,ensembl_havana,gene,448948,453948,+,ENSG00000237094,RP4-669L17.10
4,1,havana,gene,536816,541816,-,ENSG00000230021,RP5-857K21.4
...,...,...,...,...,...,...,...,...
13549,21,ensembl_havana,gene,47674304,47679304,+,ENSG00000215424,MCM3AP-AS1
13550,21,ensembl_havana,gene,47712665,47717665,+,ENSG00000182362,YBEY
13551,21,ensembl_havana,gene,47720095,47725095,-,ENSG00000160298,C21orf58
13552,21,ensembl_havana,gene,47984926,47989926,+,ENSG00000160305,DIP2A


In [ ]:
del Plus_filtered_DF, Plus_Nend_col, Minus_filtered_DF, Minus_Nstart_col, UTES_DF

# For finding intergenics region, apply the below code for acquiring whole genes including the regulatory sites as well.

In [ ]:
del fl3, rline, column

Extend genes toward their TSS region for discluing regulatory region before attaining intergenic regions.

In [ ]:
fl3 = open("/content/drive/MyDrive/Scripts/Data/Filtered_sorted_WGS_GeneAnnotation.csv", "r")

fl3_list = []

for rline in fl3:
  column = rline.split("\t")

  if column[5] == "+":
    column[3] = int(column[3]) - 5000
    fl3_list.append([column[0], column[3], column[4], column[5], column[7]])

  elif column[5] == "-":
    column[4] = int(column[4]) + 5000
    fl3_list.append([column[0], column[3], column[4], column[5], column[7]])

fl3.close()

In [ ]:
del fl3, rline, column

In [ ]:
fl3_DF = pd.DataFrame(fl3_list, columns= ["Chromosome_Name", "Start", "End", "Strand", "Gene_Name"])

In [ ]:
del fl3_list

In [ ]:
fl3_DF

Remove unnecessary characters from the dataframe, if exist.


In [ ]:
fl3_DF["Gene_Name"] = fl3_DF["Gene_Name"].str.replace('"', '').str.replace('\\n$','')

In [ ]:
fl3_DF

Add "chr" suffix to the beginning of  chromosome name.

In [ ]:
fl3_DF["Chromosome_Name"] = fl3_DF["Chromosome_Name"].apply(lambda x: f"chr{x}")

In [ ]:
fl3_DF

,Chromosome_Name,Start,End,Strand,Gene_Name
0,chr1,14363,34807,-,WASH7P
1,chr1,141474,178863,-,RP11-34P13.13
2,chr1,227615,272254,-,AP006222.2
3,chr1,312719,453948,+,RP4-669L17.10
4,chr1,536816,664931,-,RP5-857K21.4
...,...,...,...,...,...
13169,chr21,47644130,47679304,+,MCM3AP-AS1
13170,chr21,47701250,47717665,+,YBEY
13171,chr21,47720095,47748790,-,C21orf58
13172,chr21,47873811,47989926,+,DIP2A


# Discard genes coordinates, which are excluded in 5kb divided reference genome and create regulatory sites extended genes.

Do not forget to change the distance between adjacent genes as 15000, instead of 10000.

In [ ]:
Ref_Gen_DF = pd.read_csv("/content/drive/MyDrive/Scripts/Data/5kb_divided_hg19.bed", delimiter="\t", header=None)
Ref_Gen_DF

In [ ]:
Ref_Gen_DF[0] = Ref_Gen_DF[0].str.replace('chrM', 'chrY')

Create a list of chromosome numbers

In [ ]:
chr_name_list = Ref_Gen_DF[0].unique().tolist()
chr_name_list

Create a dictionary in which keys are chromosome names and values are number of genes for each given chromosome.

In [ ]:
index_num_dict = {}

for loop in chr_name_list:
  index_num = len(Ref_Gen_DF[Ref_Gen_DF[0] == loop])
  index_num_dict[loop] = index_num

print(index_num_dict)

{'chr1': 49792, 'chr10': 26760, 'chr11': 27018, 'chr12': 26656, 'chr13': 22873, 'chr14': 21409, 'chr15': 20399, 'chr16': 18068, 'chr17': 16652, 'chr18': 16075, 'chr19': 11724, 'chr2': 48439, 'chr20': 12889, 'chr21': 9342, 'chr22': 10164, 'chr3': 39660, 'chr4': 38043, 'chr5': 36308, 'chr6': 34162, 'chr7': 31870, 'chr8': 29028, 'chr9': 27679, 'chrY': 4, 'chrX': 31209}


In [ ]:
del loop, index_num

Store the last element of each chromosome group by adding number of genes (rows) of every chromosome iteratively until the given chromosome's last element.

In [ ]:
total = 0
end_coord_list = []

for dict_key, dict_value in index_num_dict.items():
  total = total + dict_value
  if dict_key == "chr1":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr2":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr3":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr4":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr5":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr6":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr7":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chrX":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr8":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr9":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr10":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr11":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr12":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr13":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr14":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr15":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr16":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr17":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr18":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr20":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr19":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chrY":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr22":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])
  if dict_key == "chr21":
    end_coord_list.append(Ref_Gen_DF.loc[total - 1, 2])


In [ ]:
fl4_DF = fl3_DF.copy()

In [ ]:
fl4_DF

According to number of elements of each chromosome group, only look at end coordinates of the last element of each chromosome group and compare it with the last element of the each chromosome's groups' reference genome. Then discard the genes which exceed the end coordinates of the reference genome using dataframe index.

In [ ]:
for n, i in zip(chr_name_list, end_coord_list):
  print(n)
  Exceeded_indices = fl3_DF[(fl3_DF["Chromosome_Name"] == n) & (pd.to_numeric(fl3_DF["End"]) >= i)].index
  fl4_DF = fl4_DF.drop(Exceeded_indices)

In [ ]:
fl4_DF

In [ ]:
del dict_key, dict_value, total, i, n, end_coord_list, Exceeded_indices, fl3_DF

Write down Dataframe as ".BED" file for retrieving intergenic file.

In [ ]:
fl4_DF.to_csv("/content/drive/MyDrive/Scripts/Data/Extended_geneAnnotation.bed", sep='\t', index=False, header=False)

# Substract gene locations from referece genome

In [ ]:
!bedtools subtract -a '/content/drive/MyDrive/Scripts/Data/5kb_divided_hg19.bed' -b '/content/drive/MyDrive/Scripts/Data/Extended_geneAnnotation.bed' > "/content/drive/MyDrive/Scripts/Data/intergenes_init.bed"

In [ ]:
Intergenes_DF = pd.read_csv("/content/drive/MyDrive/Scripts/Data/intergenes_init.bed", delimiter="\t", names=["Chromosome_Name", "Start", "End", "S1", "S2", "S3"])

Intergenes_DF

Prepare intergene candidates regions for _bedtools intersect_.

In [ ]:
Intergenes_DF["Chromosome_Name"] = Intergenes_DF["Chromosome_Name"].str.replace("chrM","chrY")
Intergenes_DF = Intergenes_DF.drop(Intergenes_DF[Intergenes_DF["Chromosome_Name"] == "chrY"].index)

In [ ]:
Intergenes_DF

In [ ]:
Intergenes_DF[Intergenes_DF["Chromosome_Name"] == "chrY"]

Then, check for the gene start sites which had minus value.

In [ ]:
Intergenes_DF[pd.to_numeric(Intergenes_DF["Start"]) < 0]

,Chromosome_Name,Start,End,S1,S2,S3


To remove genes having minus value.

In [ ]:
target_indices = Intergenes_DF[Intergenes_DF["Start"] <= 0].index
#print(target_indices)

Intergenes_DF = Intergenes_DF.drop(target_indices)

In [ ]:
Intergenes_DF

# Select mappable intergenic regions.

In [ ]:
damage_seq = pd.read_csv("/content/drive/MyDrive/Scripts/Data/Damage_regions_Hg19Genome.bed", delimiter="\t", names=["Chromosome_Name", "Start", "End", "RPKM", "Label"])
damage_seq

## Tried to combine dataframes. - Not enough RAM

In [ ]:
Intergenes_edited_DF = pd.merge(D64A[["Chromosome_Name", "Start", "End"]], Intergenes_DF[["Chromosome_Name", "Start", "End"]], how='outer', on='Chromosome_Name', suffixes=('','_2'))
Intergenes_edited_DF

## Tried to create lists and check the coditions according to these lists.

In [ ]:
del damage_seq, D64A

In [ ]:
D64A = damage_seq[damage_seq["Label"] == "HDA64A1_ATCACG"]

In [ ]:
dmg_chr = D64A["Chromosome_Name"].to_list()
dmg_str = D64A["Start"].to_list()
dmg_end = D64A["End"].to_list()

inter_chr = Intergenes_DF["Chromosome_Name"].to_list()
inter_str = Intergenes_DF["Start"].to_list()
inter_end = Intergenes_DF["End"].to_list()

gene_indexes = []

for i in range(0, len(Intergenes_DF)):
  for j in range(0, len(D64A)):
    if inter_chr[i] == dmg_chr[j]:
      if (inter_str[i] >= dmg_str[j]) and (inter_end[i] <= dmg_end[j]):
        gene_indexes.append(i)          

In [ ]:
del i, j, inter_end, inter_str, inter_chr, dmg_end, dmg_str, dmg_chr

## Intersect damage-seq and Intergenes using _bedtools intersect_

Chaning overlap percentage does not make any change in the count values. I have tried -f values as 0.2, 0.5, and 0.9, all gave the same results.



*   There is not count value higher than 1.
*   The number of count numbers for 1 is same for all three -f values.
*   The number of count numbers for 1 is same for all three -f values.





In [ ]:
D64A = damage_seq[damage_seq["Label"] == "HDA64A1_ATCACG"]
D64A = D64A.drop(D64A[D64A["Chromosome_Name"] == "chrM"].index)

In [ ]:
D64A.to_csv("/content/drive/MyDrive/Scripts/Data/DMG64PP_RepA.bed", sep='\t', index=False, header=False)
Intergenes_DF.to_csv("/content/drive/MyDrive/Scripts/Data/intergenes_sec.bed", sep='\t', index=False, header=False)

In [ ]:
!sort -k1,1 -V -k2,2n "/content/drive/MyDrive/Scripts/Data/DMG64PP_RepA.bed" > "/content/drive/MyDrive/Scripts/Data/DMG64PP_RepA_sorted.bed"
!sort -k1,1 -V -k2,2n "/content/drive/MyDrive/Scripts/Data/intergenes_sec.bed" > "/content/drive/MyDrive/Scripts/Data/intergenes_sec_sorted.bed"

tcmalloc: large alloc 2806611968 bytes == 0x557436f04000 @  0x7f7d141631e7 0x55743451c718 0x55743451b5a1 0x7f7d13b41bf7 0x55743451c02a
tcmalloc: large alloc 1211080704 bytes == 0x55f96e8a6000 @  0x7f44b2fc01e7 0x55f96d169718 0x55f96d1685a1 0x7f44b299ebf7 0x55f96d16902a


In [ ]:
!bedtools intersect -sorted -a "/content/drive/MyDrive/Scripts/Data/intergenes_sec_sorted.bed" -b "/content/drive/MyDrive/Scripts/Data/DMG64PP_RepA_sorted.bed" -wa -c -f 1.0 > '/content/drive/MyDrive/Scripts/Data/intergenic_sites.bed'

In [ ]:
Intergenes = pd.read_csv("/content/drive/MyDrive/Scripts/Data/intergenic_sites.bed", delimiter="\t", names=["Chromosome_Name", "Start", "End", "S1", "S2", "S3", "Counts"])
Intergenes

,Chromosome_Name,Start,End,S1,S2,S3,Counts
0,chr1,0,5000,.,.,.,0
1,chr1,5000,10000,.,.,.,0
2,chr1,10000,14363,.,.,.,1
3,chr1,34807,35000,.,.,.,0
4,chr1,35000,40000,.,.,.,0
...,...,...,...,...,...,...,...
418511,chrX,156020000,156025000,.,.,.,0
418512,chrX,156025000,156030000,.,.,.,0
418513,chrX,156030000,156035000,.,.,.,0
418514,chrX,156035000,156040000,.,.,.,0


In [ ]:
Intergenes[Intergenes["Counts"] > 0]

,Chromosome_Name,Start,End,S1,S2,S3,Counts
2,chr1,10000,14363,.,.,.,1
8,chr1,55000,60000,.,.,.,1
9,chr1,60000,65000,.,.,.,1
10,chr1,65000,70000,.,.,.,1
11,chr1,70000,75000,.,.,.,1
...,...,...,...,...,...,...,...
418308,chrX,154905000,154910000,.,.,.,1
418309,chrX,154910000,154915000,.,.,.,1
418310,chrX,154915000,154920000,.,.,.,1
418311,chrX,154920000,154925000,.,.,.,1


In [ ]:
Intergenes[Intergenes["Counts"] > 1]

In [ ]:
del Intergenes

## Tried to check the conditions using series and dataframes directly. - Too slow

In [ ]:
it = 0

for i in range(0, len(Intergenes_DF)):
  for j in range(0, len(damage_seq)):
    if (Intergenes_DF.loc[i, "Chromosome_Name"] == damage_seq.loc[j, "Chromosome_Name"]) and (Intergenes_DF.loc[i, "Start"] >= damage_seq.loc[j, "Start"]) and (Intergenes_DF.loc[i, "End"] <= damage_seq.loc[j, "End"]):
      it += 1

print(it)

## Tried to use numpy condition to check and compare two dataframes. - Not sure whether its working

In [ ]:
for i in range(0, len(Intergenes_DF)):
  for j in range(0, len(damage_seq)):
    if Intergenes_DF.loc[i, "Chromosome_Name"] == damage_seq.loc[j, "Chromosome_Name"]:
      Intergenes_DF.loc[i, "Overlap"] = np.where((Intergenes_DF.loc[i, "Chromosome_Name"] == damage_seq.loc[j, "Chromosome_Name"]) & (Intergenes_DF.loc[i, "Start"] >= damage_seq.loc[j, "Start"]) & (Intergenes_DF.loc[i, "End"] <= damage_seq.loc[j, "End"]), True, False)

In [ ]:
del Intergenes_DF

# Write down genes and intergenes reads as ".bed".

In [ ]:
path = "/content/drive/MyDrive/Scripts/Data/"

UTSS_sorted_DF.to_csv(path + "U_TSS_raw.bed", sep='\t', index=False, header=False)

DTSS_sorted_DF.to_csv(path + "D_TSS_raw.bed", sep='\t', index=False, header=False)

UTES_sorted_DF.to_csv(path + "U_TES_raw.bed", sep='\t', index=False, header=False)

Intergenes_DF.to_csv(path + "Intergenes_raw.bed", sep='\t', index=False, header=False)